Parameters

In [1]:
# Choose ticker to pull 
ticker = "IBM"

In [2]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
from pandas_datareader import DataReader as pdr
import yfinance as yf
yf.pdr_override()

In [3]:
# get monthly market excess return and risk-free rate for past 60 months from French's data library
ff = pdr('F-F_Research_Data_Factors','famafrench',start=1970)[0] / 100
ff = ff.iloc[-60:]

# pull data from yahoo for ticker
ticker = ticker.upper()
ret = yf.download(ticker, start='2017-01-01', progress=False)['Adj Close']
ret = ret.resample('M').last().pct_change()
ret.index = ret.index.to_period('M')
ret.name = "ret"

# Combine with market data
df = ff.join(ret, how="left")
df["ret"] -= df.RF
df = df[["Mkt-RF", "ret"]].reset_index()
df.columns = ["date", "mkt", "ret"]
df["date"] = df.date.astype(str)

C:\Users\kpc2\AppData\Local\Temp\ipykernel_26808\4212460753.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr('F-F_Research_Data_Factors','famafrench',start=1970)[0] / 100
C:\Users\kpc2\AppData\Local\Temp\ipykernel_26808\4212460753.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr('F-F_Research_Data_Factors','famafrench',start=1970)[0] / 100


In [4]:
# Run market model regression to get alpha and beta
result = sm.OLS(df.ret, sm.add_constant(df.mkt)).fit()
alpha = result.params["const"]*100*100
beta  = result.params["mkt"]
print(f'{ticker} Alpha (bps/month):\t{alpha:.2f}')
print(f'{ticker} Beta:\t\t{beta:.2f}')

IBM Alpha (bps/month):	-1.44
IBM Beta:		0.80


In [5]:
# Plot data
fig = px.scatter(
    df,
    x="mkt",
    y="ret",
    trendline="ols",
    hover_data=dict(ret=False, mkt=False, date=False),
    hover_name="date",
)
fig.layout.xaxis["title"] = "Market Excess Return"
fig.layout.yaxis["title"] = ticker.upper() + " Excess Return"
fig.update_traces(
    marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)
fig.update_yaxes(tickformat=".0%")
fig.update_xaxes(tickformat=".0%")
fig.show()